# Extracteurs et classificateurs à contrôle ponctuel.

Souvent, lorsque nous abordons un nouveau projet, nous sommes victimes du paradoxe du choix : nous ne savons pas par où ni comment commencer en raison de la présence de tant d'options parmi lesquelles choisir. Quel extracteur de caractéristiques est le meilleur ? Quel est le modèle le plus performant que nous puissions entraîner ? Comment devons-nous pré-traiter nos données ?

Dans cette recette, nous allons implémenter un framework qui vérifiera automatiquement les extracteurs et classificateurs de caractéristiques. L'objectif n'est pas d'obtenir le meilleur modèle possible tout de suite, mais de restreindre nos options afin que nous puissions nous concentrer sur les plus prometteurs à un stade ultérieur.


Nous utiliserons un ensemble de données appelé 17 Category Flower Dataset, disponible ici : http://www.robots.ox.ac.uk/~vgg/data/flowers/17. Cependant, une version organisée, organisée en sous-dossiers par classe, peut être téléchargée ici : https://github.com/PacktPublishing/Tensorflow-2.0-Computer-Vision-Cookbook/tree/master/ch3/recipe3/flowers17.zip. Décompressez-le à l'endroit de votre choix.



In [1]:
#!cp /content/drive/MyDrive/probabilité/flowers17.zip /content/
#! unzip flowers17.zip

Les étapes suivantes nous permettront de vérifier plusieurs combinaisons d'extracteurs de caractéristiques et d'algorithmes d'apprentissage automatique. Suivez ces étapes pour terminer cette recette

**1.** Importez les packages nécessaires :

In [2]:
import json
import os
import pathlib
from glob import glob

import h5py
import numpy as np
from sklearn.ensemble import *
from sklearn.linear_model import *
import sklearn.utils as skutils
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.tree import *
from tensorflow.keras.applications import *
from tensorflow.keras.preprocessing.image import *
from tqdm import tqdm

**2.** nous réutiliserons la classe FeatureExtractor() que nous avons dénie dans Implémentation d'un extracteur de caractéristiques à l'aide d'une recette réseau pré-entraînée, au début de ce chapitre. Consultez-le si vous voulez en savoir plus.

In [3]:
class FeatureExtractor(object):
    def __init__(self,
                 model,
                 input_size,
                 label_encoder,
                 num_instances,
                 feature_size,
                 output_path,
                 features_key='features',
                 buffer_size=1000):
      
        #Nous devons nous assurer que le chemin de sortie peut être écrit

        if os.path.exists(output_path):
            error_msg = (f'{output_path} already exists. '
                         f'Please delete it and try again.')
            raise FileExistsError(error_msg)
        # Maintenant, stockons le paramètre d'entrée en tant objet
        self.model = model
        self.input_size = input_size
        self.le = label_encoder
        self.feature_size = feature_size
        
        # self.buffer contiendra un tampon d'instances et d'étiquettes, 
        #tandis que self.current_index pointera vers le prochain emplacement
        # libre dans les ensembles de données de la base de données interne HDF5. Nous allons créer ceci maintenant :

        self.db = h5py.File(output_path, 'w')
        self.features = self.db.create_dataset(features_key,
                                               (num_instances,
                                                feature_size),
                                               dtype='float')
        
        self.labels = self.db.create_dataset('labels',
                                             (num_instances,),
                                             dtype='int')

        self.buffer_size = buffer_size
        self.buffer = {'features': [], 'labels': []}
        self.current_index = 0

    # Définir une méthode qui extraira les caractéristiques et les étiquettes d'une liste de chemins d'images et les stockera dans la base de données HDF5
    def extract_features(self,
                         image_paths,
                         labels,
                         batch_size=64,
                         shuffle=True):
        if shuffle:
            image_paths, labels = skutils.shuffle(image_paths,
                                                  labels)
            
        encoded_labels = self.le.fit_transform(labels)
        self._store_class_labels(self.le.classes_)

        # Après avoir mélangé les chemins d'images et leurs étiquettes,
        # ainsi que l'encodage et le stockage de ces derniers, nous itérerons sur des lots d'images, en les passant à travers le réseau pré-entraîné. 
        #Une fois cela fait, nous enregistrerons les fonctionnalités résultantes dans la base de données HDF5 (les méthodes d'aide que nous avons utilisées ici seront définies sous peu)

        for i in tqdm(range(0, len(image_paths), batch_size)):

            batch_paths = image_paths[i: i + batch_size]
            batch_labels = encoded_labels[i:i + batch_size]
            batch_images = []

            for image_path in batch_paths:
                image = load_img(image_path,
                                 target_size=self.input_size)
                image = img_to_array(image)
                image = np.expand_dims(image, axis=0)
                image = imagenet_utils.preprocess_input(image)

                batch_images.append(image)

            batch_images = np.vstack(batch_images)
            feats = self.model.predict(batch_images,
                                       batch_size=batch_size)

            new_shape = (feats.shape[0], self.feature_size)
            feats = feats.reshape(new_shape)
            self._add(feats, batch_labels)

        self._close()
    # Définir une méthode privée qui ajoutera des entités et des étiquettes aux jeux de données correspondants
    def _add(self, rows, labels):
        self.buffer['features'].extend(rows)
        self.buffer['labels'].extend(labels)

        if len(self.buffer['features']) >= self.buffer_size:
            self._flush()
    # Définir une méthode privée qui videra le buffer sur le disque
    def _flush(self):
        next_index = (self.current_index +
                      len(self.buffer['features']))
        buffer_slice = slice(self.current_index, next_index)
        self.features[buffer_slice] = self.buffer['features']
        self.labels[buffer_slice] = self.buffer['labels']
        self.current_index = next_index
        self.buffer = {'features': [], 'labels': []}
    # Définir une méthode privée qui stockera les étiquettes de classe dans la base de données HDF5
    def _store_class_labels(self, class_labels):

        print("class_labels=====>", class_labels)
        data_type = h5py.special_dtype(vlen=str)
        label_ds = self.db.create_dataset('label_names',
                                          (len(class_labels),),
                                          dtype=data_type)
        
        label_ds[:] = class_labels
    # Définir une méthode privée qui fermera le jeu de données HDF5
    def _close(self):
        if len(self.buffer['features']) > 0:
            self._flush()

        self.db.close()

**3.** Dénissez la taille d'entrée de tous les extracteurs de caractéristiques :

In [4]:
INPUT_SIZE = (224, 224, 3)

**4.** Définissez une fonction qui obtiendra une liste de tuples de réseaux pré-entraînés, ainsi que la dimensionnalité des vecteurs qu'ils génèrent : 

In [5]:
def get_pretrained_networks():
    return [
        (VGG16(input_shape=INPUT_SIZE,
               weights='imagenet',
               include_top=False),
         7 * 7 * 512),
        (VGG19(input_shape=INPUT_SIZE,
               weights='imagenet',
               include_top=False),
         7 * 7 * 512),
        (Xception(input_shape=INPUT_SIZE,
                  weights='imagenet',
                  include_top=False),
         7 * 7 * 2048),
        (ResNet152V2(input_shape=INPUT_SIZE,
                     weights='imagenet',
                     include_top=False),
         7 * 7 * 2048),
        (InceptionResNetV2(input_shape=INPUT_SIZE,
                           weights='imagenet',
                           include_top=False),
         5 * 5 * 1536)
    ]


**5.**Définissez une fonction qui renvoie un dict de modèles d'apprentissage automatique à vérifier :

In [6]:
def get_classifiers():
    models = {}
    models['LogisticRegression'] = LogisticRegression()
    models['SGDClf'] = SGDClassifier()
    models['PAClf'] = PassiveAggressiveClassifier()
    models['DecisionTreeClf'] = DecisionTreeClassifier()
    models['ExtraTreeClf'] = ExtraTreeClassifier()

    n_trees = 100
    models[f'AdaBoostClf-{n_trees}'] = \
        AdaBoostClassifier(n_estimators=n_trees)
    models[f'BaggingClf-{n_trees}'] = \
        BaggingClassifier(n_estimators=n_trees)
    models[f'RandomForestClf-{n_trees}'] = \
        RandomForestClassifier(n_estimators=n_trees)
    models[f'ExtraTreesClf-{n_trees}'] = \
        ExtraTreesClassifier(n_estimators=n_trees)
    models[f'GradientBoostingClf-{n_trees}'] = \
        GradientBoostingClassifier(n_estimators=n_trees)

    number_of_neighbors = range(3, 25)
    for n in number_of_neighbors:
        models[f'KNeighborsClf-{n}'] = \
            KNeighborsClassifier(n_neighbors=n)

    reg = [1e-3, 1e-2, 1, 10]
    for r in reg:
        models[f'LinearSVC-{r}'] = LinearSVC(C=r)
        models[f'RidgeClf-{r}'] = RidgeClassifier(alpha=r)

    print(f'Defined {len(models)} models.')
    return models

**6.** Dénissez le chemin d'accès au jeu de données, ainsi qu'une liste de tous les chemins d'images :

In [7]:
dataset_path = (pathlib.Path('/content')/ 'flowers17')
files_pattern = (dataset_path / 'images' / '*' / '*.jpg')
images_path = [*glob(str(files_pattern))]

**7.** Charger les étiquettes en mémoire

In [8]:
labels = []
for index in tqdm(range(len(images_path))):
    image_path = images_path[index]
    image = load_img(image_path)

    label = image_path.split(os.path.sep)[-2]
    labels.append(label)

    image.close()

100%|██████████| 1360/1360 [00:00<00:00, 9184.79it/s]


**8.** Déﬁnissez certaines variables afin de garder une trace du processus de contrôle ponctuel. final_report contiendra la précision de chaque classificateur, entraîné sur les caractéristiques produites par différents réseaux pré-entraînés. best_model, best_accuracy et best_features contiendront respectivement le nom du meilleur modèle, sa précision et le nom du réseau pré-entraîné qui a produit les caractéristiques

In [9]:
final_report = {}
best_model = None
best_accuracy = -1
best_features = None

**9** Itérer sur chaque réseau pré-entraîné, en l'utilisant pour extraire des caractéristiques des images de l'ensemble de données

In [ ]:
for model, feature_size in get_pretrained_networks():
    output_path = dataset_path / f'{model.name}_features.hdf5'
    output_path = str(output_path)
    fe = FeatureExtractor(model=model,
                          input_size=INPUT_SIZE,
                          label_encoder=LabelEncoder(),
                          num_instances=len(images_path),
                          feature_size=feature_size,
                          output_path=output_path)
  
    fe.extract_features(image_paths=images_path,
                        labels=labels)
    #Prendre 80% des données pour former, et 20% pour tester
    db = h5py.File(output_path, 'r')

    TRAIN_PROPORTION = 0.8
    SPLIT_INDEX = int(len(labels) * TRAIN_PROPORTION)

    X_train, y_train = (db['features'][:SPLIT_INDEX],
                        db['labels'][:SPLIT_INDEX])
    X_test, y_test = (db['features'][SPLIT_INDEX:],
                      db['labels'][SPLIT_INDEX:])

    classifiers_report = {
        'extractor': model.name
    }

    print(f'Spot-checking with features from {model.name}')

    #À l'aide des fonctionnalités extraites dans l'itération actuelle,
    #passez en revue tous les modèles d'apprentissage automatique, entraînez-les sur l'ensemble d'entraînement et évaluez-les sur l'ensemble de test
    for clf_name, clf in get_classifiers().items():
        try:
            clf.fit(X_train, y_train)
        except Exception as e:
            print(f'\t{clf_name}: {e}')
            continue

        predictions = clf.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)

        print(f'\t{clf_name}: {accuracy}')
        classifiers_report[clf_name] = accuracy
        #Vérifiez si nous avons un nouveau meilleur modèle. Si tel est le cas, mettez à jour les variables appropriées :
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model = clf_name
            best_features = model.name
    #Stockez les résultats de cette itération dans final_report et libérez les ressources du chier HDF5
    final_report[output_path] = classifiers_report
    db.close()

219070464/219055592 [==============================] - 2s 0us/step
class_labels=====> ['bluebell' 'buttercup' 'coltsfoot' 'cowslip' 'crocus' 'daffodil' 'daisy'
 'dandelion' 'fritillary' 'iris' 'lilyvalley' 'pansy' 'snowdrop'
 'sunflower' 'tigerlily' 'tulip' 'windflower']


100%|██████████| 22/22 [01:01<00:00,  2.78s/it]


Spot-checking with features from vgg16
Defined 40 models.
	LogisticRegression: 0.8933823529411765
	SGDClf: 0.8970588235294118
	PAClf: 0.9007352941176471
	DecisionTreeClf: 0.4411764705882353
	ExtraTreeClf: 0.41544117647058826
	AdaBoostClf-100: 0.12867647058823528
	BaggingClf-100: 0.7977941176470589
	RandomForestClf-100: 0.8014705882352942
	ExtraTreesClf-100: 0.8125
	GradientBoostingClf-100: 0.6727941176470589
	KNeighborsClf-3: 0.49264705882352944
	KNeighborsClf-4: 0.4852941176470588
	KNeighborsClf-5: 0.5110294117647058
	KNeighborsClf-6: 0.5110294117647058
	KNeighborsClf-7: 0.5073529411764706
	KNeighborsClf-8: 0.5073529411764706
	KNeighborsClf-9: 0.4852941176470588
	KNeighborsClf-10: 0.5036764705882353
	KNeighborsClf-11: 0.5073529411764706
	KNeighborsClf-12: 0.5110294117647058
	KNeighborsClf-13: 0.5
	KNeighborsClf-14: 0.5073529411764706
	KNeighborsClf-15: 0.49264705882352944
	KNeighborsClf-16: 0.4852941176470588
	KNeighborsClf-17: 0.48161764705882354
	KNeighborsClf-18: 0.4852941176470588

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


	LinearSVC-0.001: 0.9044117647058824
	RidgeClf-0.001: 0.8602941176470589


...

	LinearSVC-10: 0.9375
	RidgeClf-10: 0.9080882352941176
class_labels=====> ['bluebell' 'buttercup' 'coltsfoot' 'cowslip' 'crocus' 'daffodil' 'daisy'
 'dandelion' 'fritillary' 'iris' 'lilyvalley' 'pansy' 'snowdrop'
 'sunflower' 'tigerlily' 'tulip' 'windflower']


100%|██████████| 22/22 [00:21<00:00,  1.00it/s]


Spot-checking with features from xception
Defined 40 models.


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


	LogisticRegression: 0.5551470588235294
	SGDClf: 0.5477941176470589
	PAClf: 0.5551470588235294
	DecisionTreeClf: 0.23529411764705882
	ExtraTreeClf: 0.18382352941176472
	AdaBoostClf-100: 0.09558823529411764
	BaggingClf-100: 0.41544117647058826
	RandomForestClf-100: 0.40808823529411764
	ExtraTreesClf-100: 0.41911764705882354


**10.**Mettez à jour final_report avec les informations du meilleur modèle. Enfin, écrivez-le sur le disque

In [ ]:
final_report['best_model'] = best_model
final_report['best_accuracy'] = best_accuracy
final_report['best_features'] = best_features
with open('final_report.json', 'w') as f:
    json.dump(final_report, f)

En examinant le fichier final_report.json, nous pouvons voir que le meilleur modèle est un PAClf (PassiveAggressiveClassifier), qui a atteint une précision de 0,934 (93,4%) sur l'ensemble de test et a été formé sur les fonctionnalités que nous avons extraites d'un réseau VGG19.

Dans cette recette, nous avons développé un cadre qui nous a permis de vérifier automatiquement 40 algorithmes d'apprentissage automatique différents en utilisant les fonctionnalités produites par cinq réseaux pré-formés différents, résultant en 200 expériences. En tirant parti des résultats de cette approche, nous avons constaté que la meilleure combinaison de modèles pour ce problème particulier était un PassiveAggressiveClassifier formé sur des vecteurs produits par un réseau VGG19.

Notez que nous ne nous sommes pas concentrés sur l'obtention de performances maximales, mais plutôt sur la prise d'une décision éclairée, basée sur des preuves tangibles, sur l'endroit où passer notre temps et nos ressources si nous devions optimiser un classificateur sur cet ensemble de données. Maintenant, nous savons que le réglage précis d'un classificateur passif agressif sera très probablement payant. Combien de temps nous aurait-il fallu pour arriver à cette conclusion ? Des heures ou peut-être des jours.

Le pouvoir de laisser l'ordinateur faire le gros du travail est que nous n'avons pas à deviner et, en même temps, sommes libres de consacrer notre temps à d'autres tâches. C'est super, n'est-ce pas ?
